#### Set up keys and object
To run this example, you need an api key and org key. To run the whole example it costs 6K to 7K tokens.

In [12]:
execfile('./lib.py')
import os

removeEnd = { "50256": -100 }
api_key = open_file('api_key.txt')
org_key = open_file('org_key.txt')
model = "text-davinci-003"

gptComp = GptCompletion(api_key, org_key, model)
imageCreator = ImageCreation(api_key, org_key)

#### API Call Configurations
Completions have a habit of ending prematurely so this is configured to prefer not to use the ending token.

In [13]:
requestConfig = {
    "max_tokens": 200,
    "temperature": 1,
    "logit_bias": removeEnd
}

improvementConfig = {
    "max_tokens": 100
}

pickConfig = {
    "max_tokens": 100,
    "logit_bias": removeEnd
}

#### Prompts
**obj:** the subject of the image, the user input to the program. In this example it is a nurse  
**examples:** Currently the model needs a few examples, more powerful nlp models in the future may not need this.

In [14]:
obj = "nurse"

def requestPrompt(ex):
    return \
        "Write a few prompts for a text to image model that describes a " + obj + " \n\
        This prompt must be diverse in its output. \n\
        Try to include these ideas in the prompts: diversity, specificity, imagination, detail, and emotion.\n\
        Use language that paints a picture.\n\
        The example prompts are incomplete, but the new prompts are filled in.\n\n\
        Example Prompts:" + concatList('', ex, '\n- ')  + "\n\nNew Prompts:"

examples = [
    "A male nurse of asian descent is checking the vital signs of a patient in a hospital",
    "A female nurse of african descent is administering medication to a patient in a hospital bed",
    "A male nurse of hispanic heritage is comforting a child patient in a pediatric unit"
]

def improvementPrompt(p):
    return \
        "How would you improve these prompts for a text to image model?\n \
        Include these ideas: diversity, specificity, imagination, detail, and emotion.\n \
        Keep the prompt as short as possible.\n \
        Promts:\n" \
        + p + "\n\nBEGIN IMPROVED PROMPTS\n"

def pickPrompts(prompts):
    return \
        "This is a list of prompts for a text to image model.\n\
        Pick the top 3 prompts from this list:\n" + prompts + \
        "\nBest prompts:\n"

#### Generate prompts
This will create 5 to 10 prompts for Dall-e.

In [15]:
res = gptComp.complete(requestPrompt(examples), config=requestConfig)
print(res)


- A female nurse of caucasian background is gently placing her hand on an elderly patient's shoulder in a nursing home
- In a hospital trauma unit, a male nurse of native american descent is urgently tending to a critically injured patient
- A female nurse of east-asian origin is intently observing a patient's monitoring equipment in an intensive care unit
- A male nurse of middle-eastern descent is sharing a kind word with a cancer patient in an oncology ward
- In a specialized children's hospital, a female nurse of south-asian heritage is helping to cheer up a sick child with a present
- A male nurse of african descent is tending to a new mother in the maternity ward with a gentle smile on his face. 
- In the emergency room, a female nurse of mixed race is swiftly and attentively attending to a patient with a life-threatening injury 
- A male nurse of mediterranean descent is consoling a termin


#### Pick the top results
Use GPT3 to pick the best results.

In [16]:
res = gptComp.complete(pickPrompts(res), config=pickConfig)
print(res)

1. A female nurse of caucasian background is gently placing her hand on an elderly patient's shoulder in a nursing home.
2. In a hospital trauma unit, a male nurse of native american descent is urgently tending to a critically injured patient.
3. In a specialized children's hospital, a female nurse of south-asian heritage is helping to cheer up a sick child with a present. 
 
 
 
 
 
 
 
 


#### Ask to do better
Generally, the longer GPT3 has to think about something, the better the end product is.

In [17]:
res = gptComp.complete(improvementPrompt(res), config=improvementConfig)
print(res)

1. A female nurse of caucasian descent with a kind smile warmly puts her arm around an elderly patient in a nursing home. 
2. In a hustling and bustling hospital trauma unit, a male nurse of native american descent intently works to save the life of a critically injured patient. 
3. In a solemn children's hospital, a female nurse of south-asian heritage gifts a sick child with a toy to bring them joy and comfort.


#### Create the image
Each prompt should be in its own line, this will generate the last image in the list using the Dall-e api. Image names are made from the uuid of the download url.

In [19]:
if not os.path.exists('images'):
    os.makedirs('images')

l = res.split('\n')
imageCreator.download('images', l[2])

Image downloaded to: 9a8a5e05-7219-35d2-b1e8-68abb2fa0148.png


### Put it all together
A simple function to create 4 prompts using the method above.

In [20]:
def getBestPrompts(ex):
    promptResults = gptComp.complete(requestPrompt(ex), requestConfig)
    res = gptComp.complete(pickPrompts(promptResults), config=pickConfig)
    res = gptComp.complete(improvementPrompt(res), improvementConfig)
    return reformatPrompts(res)

bp = getBestPrompts(examples)
print(bp)

 - A Native American female nurse is tending to an IV drip in a cheery hospital room.
 - An elderly European nurse is cradling a child's hand with love in a pediatric unit.
 - A young African American male nurse wheels a patient to surgery with a worried expression.
 - A South Asian female nurse is carefully taking a patient's vital signs with comforting movements.



#### Bulk create
This is another simple function that will create n prompts.

In [22]:
def get_prompts(num):
    l = []
    ex = examples
    for _ in range(int(num/3)):
        bp = getBestPrompts(ex)
        for i in bp.split('-'):
            l.append(i)
    for p in l:
        if p == ' ':
            l.remove(p)
    return l

p = get_prompts(10)
for i in p:
    print(i)

 A proud indigenous female nurse carefully examines an elderly patient in a sun
soaked medical ward with ivy
covered walls.
 
 A knowledgeable Middle Eastern male nurse takes the blood pressure of a struggling patient in an urgent care unit.
 
 A passionate Caribbean female nurse comforts a young patient with a warm smile in a bustling urban clinic.

 Reverently, a female Indigenous nurse carefully attends to an elderly patient in a hospice center.
 
 With compassion, a male African American nurse brings relief and comfort to a patient with a therapeutic massage in an intensive care unit.
 
 Caringly, a female Irish nurse tenderly administers a flu shot to a new mother in a birthing center.

 A female nurse of Native American descent is beaming with joy as she embraces an elderly hospice patient in a tender hug. 
 
 A male nurse of African heritage cradles a baby lovingly in his arms while taking a temperature in a birthing center. 
 
 A female nurse of European descent lovingly meets 

#### Dall-e 2
Once all the prompts have been created, send them to Dall-e to be generated.

In [23]:
if not os.path.exists('images'):
    os.makedirs('images')

for im in p:
    imageCreator.download('images', im)

Image downloaded to: 31394508-b6e5-31b6-8d80-35038f1323ab.png
Image downloaded to: d2d0b20b-a9f5-3c1c-bcb9-bbaa137a8c44.png
Image downloaded to: 7afd41ce-c989-39ec-8ba6-4f4d325d1a5c.png
Image downloaded to: 3c51b767-de9b-342e-b593-a1c6d81489a9.png
Image downloaded to: b6fa34ce-eacb-3f62-8b97-aa9c217e6a7b.png
Image downloaded to: 5a5aa4da-2da8-3ff2-8971-a6b91d031333.png
Image downloaded to: ccf7533b-fdc5-3ec1-96f6-ca5b35b0b499.png
Image downloaded to: 3913d9e3-bf08-3ca3-b139-08538c4c1100.png
Image downloaded to: b268adb9-cea9-37ed-b040-884bb4062059.png
Image downloaded to: 2d910ac0-fd6e-3976-808f-83f7757021b9.png
Image downloaded to: d0f2b238-37dd-3793-b121-2ae3bd71f877.png
Image downloaded to: ea7ae2af-ba1c-39de-bb2e-a09d2a62bb46.png


#### Total tokens used
This is the cost of the operation, tokens are charged per 1K tokens used, currently at .2 cents per 1K, this does not include picture generation which is about 20 cents each

In [24]:
print(gptComp.total_tokens)

6192
